## Training the Models

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
ticker_list = ['VIX', 'DXY', 'JPY', 'GBP', 'MXEU', 'MXCN']

In [13]:
# Data Preprocessing
df = pd.read_csv('FinancialMarketData.csv')

X = X = df[ticker_list]
y = df['Y']

FileNotFoundError: [Errno 2] No such file or directory: 'FinancialMarketData.csv'

In [7]:
X

,VIX,DXY,JPY,GBP,MXEU,MXCN
0,22.50,100.560,105.86,1.6460,127.75,34.30
1,21.50,101.860,105.47,1.6380,129.50,32.74
2,23.02,102.410,106.04,1.6500,126.48,32.46
3,23.45,104.920,107.85,1.6110,129.19,31.29
4,21.25,104.220,109.30,1.6110,134.67,31.32
...,...,...,...,...,...,...
1106,20.30,92.336,108.72,1.3760,140.32,110.83
1107,19.61,93.297,110.29,1.3719,142.76,108.80
1108,18.12,92.335,109.88,1.3822,144.13,111.92
1109,16.65,91.852,109.19,1.3737,144.23,108.40


In [8]:
def createMA(df, window, ticker):
    df[f'{ticker}_MA'] = df[ticker].rolling(window=window).mean()

    return df

In [9]:
for ticker in ticker_list:
    X = createMA(X, 12, ticker)

<ipython-input-8-776e46eb552b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{ticker}_MA'] = df[ticker].rolling(window=window).mean()
<ipython-input-8-776e46eb552b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{ticker}_MA'] = df[ticker].rolling(window=window).mean()
<ipython-input-8-776e46eb552b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [10]:
X

,VIX,DXY,JPY,GBP,MXEU,MXCN,VIX_MA,DXY_MA,JPY_MA,GBP_MA,MXEU_MA,MXCN_MA
0,22.50,100.560,105.86,1.6460,127.75,34.30,NaN,NaN,NaN,NaN,NaN,NaN
1,21.50,101.860,105.47,1.6380,129.50,32.74,NaN,NaN,NaN,NaN,NaN,NaN
2,23.02,102.410,106.04,1.6500,126.48,32.46,NaN,NaN,NaN,NaN,NaN,NaN
3,23.45,104.920,107.85,1.6110,129.19,31.29,NaN,NaN,NaN,NaN,NaN,NaN
4,21.25,104.220,109.30,1.6110,134.67,31.32,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1106,20.30,92.336,108.72,1.3760,140.32,110.83,22.909167,90.787917,105.673333,1.379792,136.815833,117.920833
1107,19.61,93.297,110.29,1.3719,142.76,108.80,22.431667,91.109667,106.310000,1.380508,137.647500,117.710000
1108,18.12,92.335,109.88,1.3822,144.13,111.92,21.997500,91.296500,106.793333,1.382158,138.370000,117.633333
1109,16.65,91.852,109.19,1.3737,144.23,108.40,21.448333,91.409333,107.235833,1.383092,139.125000,116.795000


In [11]:
# Drop rows with NaN values
X = X.dropna()  # Drop rows with NaN in X
y = y[X.index]  # Align y with X after dropping rows

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [15]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest


# Create a pipeline with PCA and Isolation Forest
forest = Pipeline([
    ('pca', PCA(n_components=0.95)),
    ('isolation_forest', IsolationForest(
        n_estimators=200,
        contamination=0.05,
        random_state=42
    ))
])

forest.fit(X_train)

Pipeline(steps=[('pca', PCA(n_components=0.95)),
                ('isolation_forest',
                 IsolationForest(contamination=0.05, n_estimators=200,
                                 random_state=42))])

In [16]:
import pickle

In [17]:
def evaluate_model(model, X_train, X_test, y_train, y_test, filename):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model.__class__.__name__} Accuracy: {accuracy:.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

    print(f"Model saved as {filename}")

In [18]:
def evaluate_isolation_forest(model, X_test, y_test, filename):
    y_pred = model.predict(X_test)
    # Convert 1 to 0 (normal) and -1 to 1 (anomaly)
    y_pred = np.where(y_pred == 1, 0, 1)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model.__class__.__name__} Accuracy: {accuracy:.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

    print(f"Model saved as {filename}")

In [19]:
evaluate_isolation_forest(forest, X_test, y_test, "forest.pkl")

Pipeline Accuracy: 0.8273

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.99      0.90       171
           1       0.92      0.24      0.39        49

    accuracy                           0.83       220
   macro avg       0.87      0.62      0.64       220
weighted avg       0.84      0.83      0.79       220

Model saved as forest.pkl


In [20]:
from imblearn.over_sampling import SMOTE

In [21]:
smote = SMOTE(random_state=42)

In [22]:
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [23]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(random_state=42)

In [24]:
evaluate_model(xgb_model, X_resampled, X_test, y_resampled, y_test, "xgboost.pkl")

XGBClassifier Accuracy: 0.8773

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       171
           1       0.75      0.67      0.71        49

    accuracy                           0.88       220
   macro avg       0.83      0.80      0.82       220
weighted avg       0.87      0.88      0.87       220

Model saved as xgboost.pkl


In [25]:
# Logisitic Regression
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [26]:
evaluate_model(lr_model, X_train, X_test, y_train, y_test, "lr_model.pkl")

LogisticRegression Accuracy: 0.8591

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       171
           1       0.76      0.53      0.63        49

    accuracy                           0.86       220
   macro avg       0.82      0.74      0.77       220
weighted avg       0.85      0.86      0.85       220

Model saved as lr_model.pkl


## Fetching Real Data from yfinance

In [27]:
import yfinance as yf

In [28]:
yf_tickers = ['^VIX', 'DX-Y.NYB', 'JPY=X', 'GBPUSD=X', 'MXEU.L', 'MXCN.SW']

tickers = yf.Tickers('^VIX DX-Y.NYB JPY=X GBPUSD=X MXEU.L MXCN.SW')
tickers.tickers['^VIX'].info
data = yf.download(yf_tickers, period='6mo')

[*********************100%***********************]  6 of 6 completed


In [29]:
# Rename columns
column_mapping = dict(zip(yf_tickers, ticker_list))  # Create a mapping dictionary
data = data.rename(columns=column_mapping)  # Apply renaming

In [30]:
data

Price            Close                                                       \
Ticker             DXY       GBP         JPY       MXCN     MXEU        VIX   
Date                                                                          
2024-07-10  105.050003       NaN         NaN  21.760000  28660.0  12.850000   
2024-07-11  104.440002  1.285264  161.606995  22.135000  28765.0  12.920000   
2024-07-12  104.089996  1.292374  158.201004  22.290001  28977.5  12.460000   
2024-07-15  104.190002  1.296680  158.313004  22.165001  28682.5  13.120000   
2024-07-16  104.269997  1.297000  158.141006  22.065001  28595.0  13.190000   
...                ...       ...         ...        ...      ...        ...   
2025-01-06  108.260002  1.242684  157.481003  23.959999  28195.0  16.040001   
2025-01-07  108.540001  1.251095  157.742996  23.844999  28225.0  17.820000   
2025-01-08  109.089996  1.247785  158.235992  23.559999  28345.0  17.700001   
2025-01-09  109.180000  1.235987  158.192001  23.559999  28585.0        NaN   
2025-01-10  109.639999  1.229937  158.048004  23.160000      NaN  19.540001   

Price             High                                   ...        Open  \
Ticker             DXY       GBP         JPY       MXCN  ...         JPY   
Date                                                     ...               
2024-07-10  105.169998       NaN         NaN  21.760000  ...         NaN   
2024-07-11  105.000000  1.294867  161.746994  22.135000  ...  161.606995   
2024-07-12  104.550003  1.299207  159.376007  22.290001  ...  158.201004   
2024-07-15  104.320000  1.299545  158.358002  22.165001  ...  158.313004   
2024-07-16  104.510002  1.298027  158.841003  22.065001  ...  158.141006   
...                ...       ...         ...        ...  ...         ...   
2025-01-06  109.070000  1.255067  157.949997  23.959999  ...  157.481003   
2025-01-07  108.699997  1.257640  158.421005  23.875000  ...  157.742996   
2025-01-08  109.379997  1.249407  158.542999  23.559999  ...  158.235992   
2025-01-09  109.379997  1.236644  158.304001  23.559999  ...  158.192001   
2025-01-10  109.966003  1.232195  158.856995  23.160000  ...  158.048004   

Price                                     Volume                             \
Ticker           MXCN     MXEU        VIX    DXY  GBP  JPY    MXCN     MXEU   
Date                                                                          
2024-07-10  21.760000  28660.0  12.510000    0.0  NaN  NaN     0.0      0.0   
2024-07-11  22.135000  28770.0  12.880000    0.0  0.0  0.0     0.0    441.0   
2024-07-12  22.290001  28977.5  12.870000    0.0  0.0  0.0     0.0      0.0   
2024-07-15  22.165001  28735.0  12.780000    0.0  0.0  0.0     0.0      1.0   
2024-07-16  22.065001  28585.0  13.380000    0.0  0.0  0.0     0.0      2.0   
...               ...      ...        ...    ...  ...  ...     ...      ...   
2025-01-06  23.959999  28075.0  16.770000    0.0  0.0  0.0     0.0      2.0   
2025-01-07  23.875000  28205.0  16.480000    0.0  0.0  0.0  1240.0  12903.0   
2025-01-08  23.475000  28290.0  17.910000    0.0  0.0  0.0  2593.0    437.0   
2025-01-09  23.559999  28585.0        NaN    0.0  0.0  0.0     0.0      0.0   
2025-01-10  23.160000      NaN  18.290001    0.0  0.0  0.0     0.0      NaN   

Price            
Ticker      VIX  
Date             
2024-07-10  0.0  
2024-07-11  0.0  
2024-07-12  0.0  
2024-07-15  0.0  
2024-07-16  0.0  
...         ...  
2025-01-06  0.0  
2025-01-07  0.0  
2025-01-08  0.0  
2025-01-09  NaN  
2025-01-10  0.0  

[132 rows x 30 columns]

In [31]:
data = data[['Close']]

In [32]:
data = data[['Close']].droplevel(level=0, axis=1)

In [33]:
for ticker in ticker_list:
    data = createMA(data, 12, ticker)

In [34]:
data

Ticker,DXY,GBP,JPY,MXCN,MXEU,VIX,VIX_MA,DXY_MA,JPY_MA,GBP_MA,MXEU_MA,MXCN_MA
Date,,,,,,,,,,,,
2024-07-10,105.050003,NaN,NaN,21.760000,28660.0,12.850000,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-11,104.440002,1.285264,161.606995,22.135000,28765.0,12.920000,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-12,104.089996,1.292374,158.201004,22.290001,28977.5,12.460000,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-15,104.190002,1.296680,158.313004,22.165001,28682.5,13.120000,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-16,104.269997,1.297000,158.141006,22.065001,28595.0,13.190000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-06,108.260002,1.242684,157.481003,23.959999,28195.0,16.040001,NaN,NaN,157.119415,1.251949,NaN,NaN
2025-01-07,108.540001,1.251095,157.742996,23.844999,28225.0,17.820000,NaN,NaN,157.375248,1.251476,NaN,NaN
2025-01-08,109.089996,1.247785,158.235992,23.559999,28345.0,17.700001,NaN,NaN,157.424581,1.251308,NaN,NaN


In [35]:
data = data.dropna()

In [36]:
# Reset the index to make 'Date' a regular column
data = data.reset_index()

# Remove the 'Date' column
data = data.drop('Date', axis=1)

In [37]:
data

Ticker,DXY,GBP,JPY,MXCN,MXEU,VIX,VIX_MA,DXY_MA,JPY_MA,GBP_MA,MXEU_MA,MXCN_MA
0,104.320000,1.285678,153.697006,21.184999,28487.5,16.389999,15.095000,104.261666,157.093086,1.293115,28512.916667,21.808333
1,104.559998,1.286836,154.139008,21.070000,28352.5,16.600000,15.401667,104.271666,156.470754,1.293246,28478.541667,21.719583
2,104.550003,1.286041,153.832993,20.950001,28485.0,17.690001,15.837500,104.309999,156.106753,1.292718,28437.500000,21.607917
3,104.099998,1.283697,152.669998,21.350000,28712.5,16.360001,16.107500,104.302499,155.636503,1.291636,28440.000000,21.540000
4,101.889999,1.294716,147.955994,21.674999,28917.5,14.650000,21.781667,102.835000,147.043832,1.278681,28206.875000,21.222916
...,...,...,...,...,...,...,...,...,...,...,...,...
61,106.959999,1.269036,154.085999,24.665001,28077.5,15.870000,13.886667,106.490000,151.286917,1.271123,28313.541667,24.766667
62,108.029999,1.271084,153.572006,24.719999,28077.5,27.620001,15.076667,106.621667,151.593085,1.271200,28309.375000,24.804167
63,108.410004,1.256771,154.673004,24.665001,27685.0,24.090000,15.975833,106.791667,152.023502,1.270443,28255.833333,24.824584
64,107.620003,1.249797,157.643997,24.665001,27445.0,18.360001,16.385000,106.898334,152.689919,1.268974,28181.250000,24.858334


## Testing with yfinance Data

In [52]:
import pickle
import pandas as pd
import numpy as np

# Function to load model
def load_model(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

# Load Models
xgboost_model = load_model('xgboost.pkl')
lr_model = load_model('lr_model.pkl')
forest = load_model('forest.pkl')

# Preprocess given data into a dataframe
def preprocess_data(input_dict):
    # Extract relevant features from the input dictionary
    features = ['VIX', 'DXY', 'JPY', 'GBP', 'MXEU', 'MXCN']
    data = {feature: [input_dict.get(feature, None)] for feature in features}  # Use get to handle missing keys

    # Calculate moving averages (MA) for each ticker
    window = 12  # Adjust window size as needed
    for ticker in features:
        data[f'{ticker}_MA'] = [input_dict.get(f'{ticker}_MA', None)]  # Assume MA is provided, or calculate it if needed

    # Create a pandas DataFrame
    df = pd.DataFrame(data)
    return df


# Get predictions and probabilities from all models and return average
def get_prediction(transaction_dict):
    preprocessed_data = preprocess_data(transaction_dict)

    # Get predictions from all models
    predictions = {
        'XGBoost': xgboost_model.predict(preprocessed_data)[0],
        'Logistic Regression': lr_model.predict(preprocessed_data)[0],
        'Isolation Forest': forest.predict(preprocessed_data)[0]
    }

    # Get probabilities from models that support predict_proba
    probabilities = {
        'XGBoost': xgboost_model.predict_proba(preprocessed_data)[0][1],
        # 'Logistic Regression': lr_model.predict_proba(preprocessed_data)[0][1],
        # 'Isolation Forest': forest.predict_proba(preprocessed_data)[0][1]
    }

    return predictions, probabilities


def predict(data: dict):
    prediction, probabilities = get_prediction(data)

    # Convert NumPy types to Python types
    prediction = {model: int(pred) for model, pred in prediction.items()}
    probabilities = {model: float(prob) for model, prob in probabilities.items()}

    return {
        "prediction": prediction,
        "probability": probabilities
    }

In [53]:
data.iloc[7].to_dict()

{'DXY': 101.51000213623047,
 'GBP': 1.309346079826355,
 'JPY': 145.1179962158203,
 'MXCN': 21.334999084472656,
 'MXEU': 28862.5,
 'VIX': 17.549999237060547,
 'VIX_MA': 18.450833479563396,
 'DXY_MA': 102.4283332824707,
 'JPY_MA': 146.75958251953125,
 'GBP_MA': 1.2854622701803844,
 'MXEU_MA': 28489.583333333332,
 'MXCN_MA': 21.299583117167156}

In [7]:
predict(data.iloc[29].to_dict())

NameError: name 'predict' is not defined